# **Lab 13: Home Run Derby**

In [ ]:

from pylab import *
from random import uniform
from ipywidgets import interact, fixed, Dropdown 

## Same code as lab 10, but we add widgets

In [53]:
# constants/variables 
GRAVITY = 9.81  # m/s^2
LENGTH = 100  # m
TOLERANCE = 1
LOWER_BOUND = LENGTH - TOLERANCE
UPPER_BOUND = LENGTH + TOLERANCE
WALL_HEIGHT = 8  # m

velocity: float
angle: float

# map of states
dict = {0: 'The ball hit the wall!', 1: 'Home run!!', 2: 'The ball hit the ground.'}

# function for duration of projectile motion
def t_flight(velocity, angle) -> float:
    return 2*velocity*sin(radians(angle))/GRAVITY

# function to check in which state the ball is in
def checkState(x, y, dict) -> str:
    # check if the ball hit the wall
    if (LOWER_BOUND <= x <= UPPER_BOUND) and ( 0 < y <= WALL_HEIGHT):
        print(f'x: {x:.2f}m | y: {y:.2f}m')
        return dict[0]
    # check for home run
    elif x >= LENGTH and y >= WALL_HEIGHT: 
        print(f'x: {x:.2f}m | y: {y:.2f}m')
        return dict[1]
    # ball hit the ground case
    else:
        y = 0
        if x < LENGTH:
            print(f'x: {x:.2f}m | y: {y:.2f}m')
            return dict[2]

# function to graph the motion
def graphMotion(velocity, angle):
    # figure information
    figure(figsize=(8, 6))

    # parameter vectors
    t = linspace(0, t_flight(velocity, angle), 1000)
    x = velocity*t*cos(radians(angle))
    y = velocity*t*sin(radians(angle)) - 0.5*GRAVITY*t**2

    x = [item for item in x if item <= ub]
    y = y[:len(x)]
    t = t[:len(x)]

    title(f'{checkState(x[-1], y[-1], dict)}')

    xlabel('Distance [$m$]')
    ylabel('Height [$m$]')

    # plot vertical wall at x = 100 with height 8
    plot([LENGTH, LENGTH], [0, WALL_HEIGHT], 'r-')

    # plot the trajectory
    plot(x, y)

    xlim(0,LENGTH+10)
    ylim(0,50)

    # savefig('ground.png')
    # savefig('wall.png')
    # savefig('homerun.png')

interact(graphMotion, velocity=(1., 100.), angle=(1., 89.))

interactive(children=(FloatSlider(value=50.5, description='velocity', min=1.0), FloatSlider(value=45.0, descri…

<function __main__.graphMotion(velocity, angle)>

### Extra Credit: Add a pull-down menu that enables user to select different buildings

In [ ]:

# Throw everything into a class
class Derby:
    def __init__(self):
        # Initialize with random building dimensions
        self.buildingWidth = uniform(5, 15)  # Random width between 5-15m
        self.buildingHeight = uniform(10, 30)  # Random height between 10-30m
        self.houseWidth = uniform(8, 12)  # Random width between 8-12m
        self.houseHeight = uniform(5, 8)  # Random height between 5-8m
        
    def getFlightTime(self, velocity, angle) -> float:
        """Calculate the time of flight"""
        return 2 * velocity * sin(radians(angle)) / GRAVITY
    
    def getObstacleBounds(self, obstacleType):
        """Get the x-bounds and height of the obstacle"""
        width = self.getObstacleWidth(obstacleType)
        height = self.getObstacleHeight(obstacleType)
        xStart = LENGTH - width  # Left edge of obstacle
        xEnd = LENGTH  # Right edge of obstacle
        return xStart, xEnd, height
    
    def checkState(self, x, y, obstacleType) -> str:
        """Determine what happened to the ball"""
        xStart, xEnd, obstacleHeight = self.getObstacleBounds(obstacleType)
        
        # Check if ball hit the obstacle
        if (xStart <= x <= xEnd) and (0 <= y <= obstacleHeight):
            print(f'x: {x:.2f}m | y: {y:.2f}m')
            return 'Broken Window!' if obstacleType in ['House', 'Building'] else 'The ball hit the wall!'
            
        # Check for home run (cleared the obstacle)
        elif x >= xEnd and y >= obstacleHeight:
            print(f'x: {x:.2f}m | y: {y:.2f}m')
            return 'Home run!!'
            
        # Ball hit the ground case
        else:
            if x < xEnd:
                print(f'x: {x:.2f}m | y: {0:.2f}m')
                return 'The ball hit the ground.'
            
    def getObstacleHeight(self, obstacleType):
        """Get height based on obstacle type"""
        if obstacleType == 'Wall':
            return WALL_HEIGHT
        elif obstacleType == 'House':
            return self.houseHeight
        else:  # Building
            return self.buildingHeight
            
    def getObstacleWidth(self, obstacleType):
        """Get width based on obstacle type"""
        if obstacleType == 'Wall':
            return 1  # Wall is thin
        elif obstacleType == 'House':
            return self.houseWidth
        else:  # Building
            return self.buildingWidth
    
    def drawObstacle(self, ax, obstacleType):
        """Draw the selected obstacle"""
        height = self.getObstacleHeight(obstacleType)
        width = self.getObstacleWidth(obstacleType)
        
        # Position the obstacle so its right edge is at x=LENGTH
        xStart = LENGTH - width
        
        # Create and add the rectangle
        colorMap = {
            'Wall': 'gray',
            'House': 'brown',
            'Building': 'darkgray'
        }
        
        obstacle = Rectangle((xStart, 0), width, height, 
                           facecolor=colorMap[obstacleType],
                           alpha=0.5)
        ax.add_patch(obstacle)
        
        return height  # Return height for trajectory calculations
    
    def graphMotion(self, velocity, angle, obstacleType):
        """Graph the projectile motion with current parameters"""
        # Calculate trajectory
        t = linspace(0, self.getFlightTime(velocity, angle), 1000)
        x = velocity * t * cos(radians(angle))
        y = velocity * t * sin(radians(angle)) - 0.5 * GRAVITY * t**2
        
        # Filter vectors and match lengths
        xVals = [item for item in x if item <= UPPER_BOUND]
        yVals = y[:len(xVals)]
        
        # Create figure
        fig = figure(figsize=(8, 6))
        ax = fig.gca()
        
        # Draw obstacle and get its height
        obstacleHeight = self.drawObstacle(ax, obstacleType)
        
        # Plot trajectory
        plot(xVals, yVals, 'b-', label='Ball Trajectory')
        
        # Set title based on outcome
        title(f'{self.checkState(xVals[-1], yVals[-1], obstacleType)}')
        
        # Labels and limits
        xlabel('Distance [m]')
        ylabel('Height [m]')
        xlim(0, LENGTH + 10)
        ylim(0, max(50, obstacleHeight + 10))  # Adjust ylim based on obstacle height
        
        # Add legend
        legend()
    
    def makeInteractive(self):
        """Create the interactive widget"""
        # Create dropdown for obstacle selection
        obstacleDropdown = Dropdown(
            options=['Wall', 'House', 'Building'],
            value='Wall',
            description='obstacle:'
        )
        
        # Create the interactive widget
        interact(self.graphMotion, 
                velocity=(1., 100., 1.),      # min, max, step
                angle=(1., 89., 1.),          # min, max, step
                obstacleType=obstacleDropdown)

# Create and display the widget
derby = Derby()
derby.makeInteractive()

interactive(children=(FloatSlider(value=50.0, description='velocity', min=1.0, step=1.0), FloatSlider(value=45…